<a href="https://colab.research.google.com/github/Leo140102/desafioPython/blob/main/desafioPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import hashlib
import requests
import pandas as pd
import sqlite3
from google.colab import userdata

# Chaves de API
PUBLIC_KEY = userdata.get('PUBLIC_KEY')
PRIVATE_KEY = userdata.get('PRIVATE_KEY')

# Criando o hash MD5
hashtre = hashlib.md5(str.encode(str(1) + str(PRIVATE_KEY) + str(PUBLIC_KEY)))

# Base URL da API da Marvel
base = "http://gateway.marvel.com"
url_montada_characters = f"{base}/v1/public/characters?ts=1&apikey={PUBLIC_KEY}&hash={hashtre.hexdigest()}"
url_montada_comics = f"{base}/v1/public/comics?ts=1&apikey={PUBLIC_KEY}&hash={hashtre.hexdigest()}"
url_montada_events = f"{base}/v1/public/events?ts=1&apikey={PUBLIC_KEY}&hash={hashtre.hexdigest()}"

# Fazendo a requisição GET para a API e obtendo os dados
response_characters = requests.get(url_montada_characters)
response_comics = requests.get(url_montada_comics)
response_events = requests.get(url_montada_events)

# Convertendo a resposta para JSON
dados_characters = response_characters.json()
dados_comics = response_comics.json()
dados_events = response_events.json()

# Acessando os resultados dos personagens
descricao_characters = dados_characters["data"]["results"]
descricao_comics = dados_comics["data"]["results"]
descricao_events = dados_events["data"]["results"]

In [3]:
# Conectando ao banco de dados SQLite (cria o banco se não existir)
# Usando o "with" para garantir que a conexão seja fechada automaticamente
with sqlite3.connect('marvel.db') as conn:
    cursor = conn.cursor()

    # Criando a tabela para armazenar personagens
    cursor.execute("""CREATE TABLE IF NOT EXISTS characters (
                        id INTEGER PRIMARY KEY,
                        name TEXT,
                        description TEXT)""")

    # Criando a tabela para armazenar quadrinhos
    cursor.execute("""CREATE TABLE IF NOT EXISTS comics (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        comic_title TEXT)""")

    # Criando a tabela para armazenar eventos
    cursor.execute("""CREATE TABLE IF NOT EXISTS events (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        title TEXT,
                        description TEXT,
                        start DATE,
                        end DATE)""")

In [ ]:
# Inserindo os dados dos personagens no banco de dados
    for char in descricao_characters:
        char_id = char["id"]
        char_name = char["name"]
        char_description = char["description"]
        cursor.execute("""INSERT OR IGNORE INTO characters (id, name, description)
                          VALUES (?, ?, ?)""", (char_id, char_name, char_description))

    # Inserindo os dados dos quadrinhos no banco de dados
    for comic in descricao_comics:
        comic_title = comic["title"]
        cursor.execute("""INSERT INTO comics (comic_title)
                          VALUES (?)""", (comic_title,))

    # Inserindo os dados dos eventos no banco de dados
    for even in descricao_events:
        even_title = even["title"]
        even_description = even["description"]
        even_start = even["start"]
        even_end = even["end"]
        cursor.execute("""INSERT OR IGNORE INTO events (title, description, start, end)
                          VALUES (?, ?, ?, ?)""", (even_title, even_description, even_start, even_end))